In [13]:
import pandas_datareader.data as pdr
import yfinance as yf
from BackTestAlgo import *
from Order import *
from Record import *
from functools import reduce

In [2]:
def my_init(context):
    context.symbols = ['MRNA', 'NVDA', 'AMZN', 'ARVL']
    # universe를 symbols로 정의(반드시 지정해야함)

    context.price = 'Adj Close'
    # price로 사용할 칼럼 인덱스 이름을 알려줘야함(open, high, close 등의 인덱스를 price로 활용 가능함)
    # 여기서 지정한 인덱스는 포트폴리오를 평가할 때 기본적으로 사용하는 지표임

    context.capital_base = 5000
    # 투자원금을 설정함. 지정하지 않으면 기본값을 사용함

    # context.market_benchmark = 'SP500'
    # 지정하면 win_rate 계산 시에 비교, 지정 안하면 절대수익(0%와 비교)으로 계산

    context.i = 0

In [11]:
def handle_data(context, data):
    context.i += 1
    print('\n')
    print(data.current_time)

    if context.i == 1:
        price = [data.data.loc[(data.data['date']==context.current_time)&(data.data['symbol']==symbol), 'Adj Close'].iat[0] for symbol in ['NVDA', 'AMZN']]
        order(context, ['NVDA', 'AMZN'], price, [10, 10])
    
    if context.i == 2:
        deposit(context, 5000)
        
    if context.i == 3:
        price = [data.data.loc[(data.data['date']==context.current_time)&(data.data['symbol']==symbol), 'Adj Close'].iat[0] for symbol in ['NVDA', 'AMZN']]
        order(context, ['NVDA', 'AMZN'], price, [-10, -10])
    
    nvda_price = data.data.loc[(data.data['date']==context.current_time)&(data.data['symbol']=='NVDA'), 'Adj Close'].iat[0]
    amzn_price = data.data.loc[(data.data['date']==context.current_time)&(data.data['symbol']=='AMZN'), 'Adj Close'].iat[0]
    
    record(context, nvda=nvda_price, amzn=amzn_price)

In [4]:
yf.pdr_override()

start_date = '25-05-2021'
end_date = '18-07-2021'

start = datetime.datetime.strptime(start_date, '%d-%m-%Y')
end = datetime.datetime.strptime(end_date, '%d-%m-%Y')

temp_data = pdr.get_data_yahoo('^KS11', data_source='yahoo', start=start, end=end)

temp_data['date'] = temp_data.index
temp_data = temp_data.reset_index(drop=True)
temp_data = temp_data.rename(columns={'Adj Close': 'price'})
temp_data['benchmark'] = 'kospi_200'
benchmark = temp_data

[*********************100%***********************]  1 of 1 completed


In [5]:
mrna = pdr.get_data_yahoo('mrna', data_source='yahoo', start=start, end=end)
nvda = pdr.get_data_yahoo('nvda', data_source='yahoo', start=start, end=end)
amzn = pdr.get_data_yahoo('amzn', data_source='yahoo', start=start, end=end)
arvl = pdr.get_data_yahoo('arvl', data_source='yahoo', start=start, end=end)

mrna['date'] = mrna.index
mrna = mrna.reset_index(drop=True)
mrna['symbol'] = 'MRNA'

nvda['date'] = nvda.index
nvda = nvda.reset_index(drop=True)
nvda['symbol'] = 'NVDA'

amzn['date'] = amzn.index
amzn = amzn.reset_index(drop=True)
amzn['symbol'] = 'AMZN'

arvl['date'] = arvl.index
arvl = arvl.reset_index(drop=True)
arvl['symbol'] = 'ARVL'

data = pd.concat([mrna, nvda, amzn, arvl])
data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,date,symbol
0,162.880997,166.800003,161.009995,164.169998,164.169998,4501300,2021-05-24,MRNA
1,166.559998,171.740005,165.100006,169.259995,169.259995,7759900,2021-05-25,MRNA
2,168.699997,175.619995,168.509995,175.589996,175.589996,6885200,2021-05-26,MRNA
3,174.199997,180.000000,173.630005,179.539993,179.539993,10844900,2021-05-27,MRNA
4,178.623001,186.639999,178.360001,185.009995,185.009995,8567200,2021-05-28,MRNA
...,...,...,...,...,...,...,...,...
33,15.550000,15.750000,14.920000,15.000000,15.000000,1510600,2021-07-12,ARVL
34,14.860000,14.930000,14.220000,14.290000,14.290000,2021100,2021-07-13,ARVL
35,14.510000,14.580000,13.310000,13.370000,13.370000,3025200,2021-07-14,ARVL
36,13.490000,13.610000,13.200000,13.510000,13.510000,3948000,2021-07-15,ARVL


In [14]:
tester = BackTester(initialize=my_init, tradingAlgo=handle_data)

result = tester.run(data, benchmark)


Hello backtesting


2021-05-24 00:00:00


2021-05-25 00:00:00


2021-05-26 00:00:00


2021-05-27 00:00:00


2021-05-28 00:00:00


2021-06-01 00:00:00


2021-06-02 00:00:00


2021-06-03 00:00:00


2021-06-04 00:00:00


2021-06-07 00:00:00


2021-06-08 00:00:00


2021-06-09 00:00:00


2021-06-10 00:00:00


2021-06-11 00:00:00


2021-06-14 00:00:00


2021-06-15 00:00:00


2021-06-16 00:00:00


2021-06-17 00:00:00


2021-06-18 00:00:00


2021-06-21 00:00:00


2021-06-22 00:00:00


2021-06-23 00:00:00


2021-06-24 00:00:00


2021-06-25 00:00:00


2021-06-28 00:00:00


2021-06-29 00:00:00


2021-06-30 00:00:00


2021-07-01 00:00:00


2021-07-02 00:00:00


2021-07-06 00:00:00


2021-07-07 00:00:00


2021-07-08 00:00:00


2021-07-09 00:00:00


2021-07-12 00:00:00


2021-07-13 00:00:00


2021-07-14 00:00:00


2021-07-15 00:00:00


2021-07-16 00:00:00


In [15]:
result.columns

Index(['date', 'capital_base', 'starting_cash', 'ending_cash',
       'starting_stock_value', 'ending_stock_value',
       'starting_portfolio_value', 'ending_portfolio_value',
       'portfolio_return', 'annualized_return', 'roll_annualized_return',
       'volatility', 'roll_volatility', 'sharp_ratio', 'roll_sharp_ratio',
       'cumulative_return', 'total_profit', 'drawdown_ratio', 'MDD',
       'underwater_period', 'absolute_win_rate', 'roll_absolute_win_rate',
       'relative_win_rate', 'roll_relative_win_rate', 'alpha', 'beta',
       'roll_alpha', 'roll_beta', 'nvda', 'amzn'],
      dtype='object')

In [17]:
pd.set_option('display.float_format', '{:.2f}'.format) # 항상 float 형식으로
col = ['date', 'capital_base', 'starting_cash', 'ending_cash',
       'starting_stock_value', 'ending_stock_value',
       'starting_portfolio_value', 'ending_portfolio_value',
       'portfolio_return', 'nvda', 'amzn']
result[col].head()

,date,capital_base,starting_cash,ending_cash,starting_stock_value,ending_stock_value,starting_portfolio_value,ending_portfolio_value,portfolio_return,nvda,amzn
0,2021-05-24,5000,5000,-29010.74,0,34010.74,5000,5000.00,0.00,156.08,3244.99
1,2021-05-25,10000,-24010.74,-24010.74,34010.74,34154.92,10000.0,10144.17,0.01,156.44,3259.05
2,2021-05-26,10000,-24010.74,10210.50,34154.92,0.00,10144.17,10210.50,0.01,156.96,3265.16
3,2021-05-27,10000,10210.5,10210.50,0.0,0.00,10210.5,10210.50,0.00,154.84,3230.11
4,2021-05-28,10000,10210.5,10210.50,0.0,0.00,10210.5,10210.50,0.00,162.41,3223.07


In [25]:
ac_df=tester.context.accounts_df
ac_df[ac_df['symbol']=='AMZN']

,date,deposit,symbol,unitPrice,amounts,withdrawal,balance
2,2021-05-24,0,AMZN,3244.99,10,32449.9,-29010.74
5,2021-05-26,0,AMZN,3265.16,-10,-32651.6,10210.5


In [39]:
import datetime as dt
ac_df.loc[ac_df['date']==dt.datetime(2021, 5, 24), 'withdrawal'].sum()

34010.7421875

In [27]:
col = ['date', 'annualized_return', 'roll_annualized_return',
       'volatility', 'roll_volatility', 'sharp_ratio', 'roll_sharp_ratio',
       'cumulative_return', 'total_profit']
result[col].head()

,date,annualized_return,roll_annualized_return,volatility,roll_volatility,sharp_ratio,roll_sharp_ratio,cumulative_return,total_profit
0,2021-05-24,0.00,NaN,0.00,NaN,0.00,NaN,0.00,0.0
1,2021-05-25,5.07,NaN,0.11,NaN,44.32,NaN,0.01,144.17
2,2021-05-26,4.75,NaN,0.09,NaN,50.80,NaN,0.02,210.5
3,2021-05-27,2.71,NaN,0.09,NaN,28.82,NaN,0.02,210.5
4,2021-05-28,1.86,NaN,0.09,NaN,20.51,NaN,0.02,210.5


In [30]:
col = ['date','drawdown_ratio', 'MDD',
       'underwater_period', 'absolute_win_rate', 'roll_absolute_win_rate',
       'relative_win_rate', 'roll_relative_win_rate', 'alpha', 'beta',
       'roll_alpha', 'roll_beta']
result[col]

,date,drawdown_ratio,MDD,underwater_period,absolute_win_rate,roll_absolute_win_rate,relative_win_rate,roll_relative_win_rate,alpha,beta,roll_alpha,roll_beta
0,2021-05-24,0.00,0.00,0 days,0.00,NaN,0.00,NaN,NaN,NaN,NaN,NaN
1,2021-05-25,0.00,0.00,0 days,0.50,NaN,0.50,NaN,NaN,NaN,NaN,NaN
2,2021-05-26,0.00,0.00,0 days,0.67,NaN,0.67,NaN,0.72,770.09,NaN,NaN
3,2021-05-27,0.00,0.00,1 days,0.50,NaN,0.50,NaN,0.01,-1.27,NaN,NaN
4,2021-05-28,0.00,0.00,2 days,0.40,NaN,0.40,NaN,0.01,-1.41,NaN,NaN
5,2021-06-01,0.00,0.00,6 days,0.33,NaN,0.33,NaN,0.01,-1.45,NaN,NaN
6,2021-06-02,0.00,0.00,7 days,0.29,NaN,0.29,NaN,0.01,-1.20,NaN,NaN
7,2021-06-03,0.00,0.00,8 days,0.25,NaN,0.25,NaN,0.01,-1.12,NaN,NaN
8,2021-06-04,0.00,0.00,9 days,0.22,NaN,0.33,NaN,0.00,-0.69,NaN,NaN
9,2021-06-07,0.00,0.00,12 days,0.20,NaN,0.30,NaN,0.00,-0.71,NaN,NaN
